In [8]:
# you must install openCV2 & cvlib
# Installing is so easy!! Go googling!!

import cv2
import time
import cvlib as cv
import os
import numpy as np
import Augmentor
import random
import shutil

In [9]:
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
    return img

def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
    return img

def brightness(img, low, high):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

In [10]:
def takePicture(path, howMany = 10, quitTime = 3):
    cnt, qCnt = 1, 0
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
    if not os.path.isdir(path):
        os.mkdir(path)
    while cnt <= howMany:
        ret, image = cap.read()
        faces, confidences = cv.detect_face(image)
        if faces == []:
            qCnt += 1
            time.sleep(1)
            if qCnt > quitTime:
                print("You Out!")
                break
            continue
        cv2.imshow("asdf", image)
        cv2.imwrite(path+str(cnt)+'.jpg', image)
        cv2.waitKey(500)
        cv2.destroyAllWindows()
        cnt += 1
    cap.release()

In [11]:
dirName = 'Bad'
path = './' + dirName + '/'
final_path = './' + dirName + '_final/'

if os.path.isdir(path):
    shutil.rmtree(path)

if os.path.isdir(final_path):
    shutil.rmtree(final_path)

os.mkdir(path)
os.mkdir(final_path)


takePicture(path)

p = Augmentor.Pipeline(path)
p.skew_top_bottom(probability = 1, magnitude=0.2)
p.process()

path_f = path + 'output_f/'; os.mkdir(path_f)
path_h = path + 'output_h/'; os.mkdir(path_h)
path_b = path + 'output_b/'; os.mkdir(path_b)


file_list = os.listdir(path)
file_list = [file for file in file_list if file.endswith('.jpg')]    

for file in file_list:
    img = cv2.imread(path+file)
    img_h = horizontal_shift(img, 0.3)
    img_f = cv2.flip(img, 1)
    img_b = brightness(img, 0.5, 1.5)
    cv2.imwrite(path_f + file + '_f.jpg', img_f)
    cv2.imwrite(path_h + file + '_h.jpg', img_h)
    cv2.imwrite(path_b + file + '_b.jpg', img_b)
    
path_list = [path, path+'output/', path_f, path_h, path_b]
aug_list = ['original', 'perspective', 'flip', 'horizontal', 'brightness']

if not os.path.isdir(final_path):
    os.mkdir(final_path)
for p, aug in zip(path_list, aug_list):
    file_list = os.listdir(p)
    file_list = [file for file in file_list if file.endswith('.jpg')]
    cnt = 1
    for file in file_list:
        shutil.copy2(p+file, final_path+str(cnt)+'_'+aug+'.jpg')
        cnt += 1

Processing <PIL.Image.Image image mode=RGB size=320x240 at 0x25C8B7B0970>: 100%|█| 10/10 [00:00<00:00, 188.66 Samples/s

Initialised with 10 image(s) found.
Output directory set to ./Bad/output.